# Distributed Training with DeepSpeed and Ray Train

This notebook will show you how to use Microsoft's DeepSpeed library with Ray Train for efficient distributed training. If you've completed the [FSDP2 tutorial](./FSDP2_RayTrain_Tutorial.ipynb), this notebook shows how to achieve similar memory optimization using DeepSpeed's ZeRO (Zero Redundancy Optimizer) technology.

In this tutorial, you:
1. Learn how DeepSpeed ZeRO partitions optimizer states, gradients, and parameters across GPUs
2. Configure DeepSpeed using JSON-like configuration dictionaries
3. Use DeepSpeed's built-in checkpointing methods
4. Compare DeepSpeed with FSDP2 to understand when to use each approach
5. Train a Vision Transformer model using DeepSpeed ZeRO Stage 2

## What is DeepSpeed?

[DeepSpeed](https://www.deepspeed.ai/) is a deep learning optimization library developed by Microsoft that makes distributed training **easy, efficient, and effective**. It's particularly known for:

> DeepSpeed ZeRO (Zero Redundancy Optimizer) partitions optimizer states, gradients, and parameters across data-parallel workers. This enables training models that are much larger than what fits on a single GPU, with minimal code changes.

Key features:
- **ZeRO (Zero Redundancy Optimizer)**: Memory optimization that partitions optimizer states, gradients, and parameters across GPUs
- **Mixed Precision Training**: FP16/BF16 training with automatic loss scaling
- **CPU/NVMe Offloading**: Extend training beyond GPU memory limits (ZeRO-Infinity)
- **Gradient Checkpointing**: Trade compute for memory
- **Sparse Attention**: Efficient attention for long sequences

## FSDP2 vs DeepSpeed: When to Use Which?

| Aspect | PyTorch FSDP2 | DeepSpeed ZeRO |
|--------|---------------|----------------|
| **Origin** | PyTorch native | Microsoft Research |
| **Ecosystem** | Tight PyTorch integration | Framework-agnostic (works with HuggingFace, Lightning, etc.) |
| **Configuration** | Python API | JSON config files |
| **CPU Offloading** | Basic support | Advanced (ZeRO-Infinity with NVMe) |
| **Optimizer Fusion** | Limited | Built-in fused optimizers |
| **Adoption** | Growing | Widely used in LLM training |
| **Best For** | PyTorch-native workflows | Large language models, HuggingFace |

**Key Insight**: DeepSpeed is the de-facto standard for training large language models (GPT, LLaMA, etc.). Understanding both FSDP2 and DeepSpeed gives you flexibility to choose the right tool for your use case.

## DeepSpeed ZeRO Stages Explained

ZeRO progressively partitions training state across GPUs:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     Memory Consumption Per GPU                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  No ZeRO     [████████████████████████████████████████] 100%            │
│  (DDP)       Parameters + Gradients + Optimizer States                  │
│                                                                         │
│  ZeRO-1      [█████████████████████████████] 75%                        │
│  (Stage 1)   Parameters + Gradients + Optimizer States (partitioned)   │
│                                                                         │
│  ZeRO-2      [██████████████████] 50%                                   │
│  (Stage 2)   Parameters + Gradients (partitioned) + Opt States (part.) │
│                                                                         │
│  ZeRO-3      [█████████] 25%                                            │
│  (Stage 3)   Parameters (partitioned) + Grads (part.) + Opt (part.)    │
│                                                                         │
│  ZeRO-Inf    [███] ~10% (with CPU/NVMe offload)                         │
│  (Infinity)  Everything offloaded when not in use                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Stage Comparison with FSDP2

| ZeRO Stage | FSDP2 Equivalent | What's Partitioned |
|------------|------------------|--------------------|
| Stage 1 | N/A (DDP-like) | Optimizer states only |
| Stage 2 | `reshard_after_forward=False` | Optimizer states + Gradients |
| Stage 3 | `reshard_after_forward=True` | Everything (params + grads + optimizer) |
| ZeRO-Infinity | `CPUOffloadPolicy()` | Stage 3 + CPU/NVMe offloading |

## Key Differences: FSDP2 vs DeepSpeed in Code

Here's a side-by-side comparison of the key changes:

### Model Wrapping

```python
# FSDP2 Approach
from torch.distributed.fsdp import fully_shard
model.to(device)
fully_shard(model, mesh=mesh, ...)  # Wrap model with FSDP2
optimizer = Adam(model.parameters())

# DeepSpeed Approach  
import deepspeed
model, optimizer, _, _ = deepspeed.initialize(
    model=model,
    config=deepspeed_config,  # JSON-like config dict
)  # Returns wrapped model AND optimizer
```

### Training Step

```python
# FSDP2 Approach
loss = criterion(model(inputs), labels)
optimizer.zero_grad()
loss.backward()           # Standard PyTorch backward
optimizer.step()

# DeepSpeed Approach
loss = criterion(model(inputs), labels)
model.backward(loss)      # DeepSpeed handles backward + zero_grad
model.step()              # DeepSpeed handles optimizer step
```

### Checkpointing

```python
# FSDP2 Approach
import torch.distributed.checkpoint as dcp
dcp.save(state_dict={"app": AppState(model, optimizer)}, ...)

# DeepSpeed Approach
model.save_checkpoint(save_dir)  # Built-in method
model.load_checkpoint(load_dir)  # Built-in method
```

## `Step 1`: Environment Setup

Let's verify our environment and install DeepSpeed. This tutorial requires:
- A Ray cluster with GPU workers (this example uses 2 GPUs)
- PyTorch 2.0+ with CUDA support
- DeepSpeed library
- Shared storage accessible from all workers (e.g., `/mnt/cluster_storage/`)

When running on open-source Ray (without Anyscale), you'll need to:
- Configure your Ray cluster manually
- Set up NFS or cloud storage for checkpointing
- Ensure PyTorch and DeepSpeed are installed on all worker nodes

In [1]:
# Check Ray cluster status
!ray status

======== Autoscaler status: 2026-02-01 17:27:29.505093 ========
Node status
---------------------------------------------------------------
Active:
 1 head_node
Idle:
 1 1xA10G:8CPU-32GB-2
 1 1xA10G:8CPU-32GB-1
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Total Usage:
 0.0/16.0 CPU
 0.0/2.0 GPU
 0.0/2.0 anyscale/accelerator_shape:1xA10G
 0.0/1.0 anyscale/cpu_only:true
 0.0/1.0 anyscale/node-group:1xA10G:8CPU-32GB-1
 0.0/1.0 anyscale/node-group:1xA10G:8CPU-32GB-2
 0.0/1.0 anyscale/node-group:head_node
 0.0/3.0 anyscale/provider:aws
 0.0/3.0 anyscale/region:us-west-2
 0B/96.00GiB memory
 0B/26.62GiB object_store_memory

Total Constraints:
 (no request_resources() constraints)
Total Demands:
 (no resource demands)


In [2]:
%%bash
pip install -q torch torchvision deepspeed

Successfully registered `torch, torchvision` and 1 other packages to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_g54aiirwj1s8t9ktgzikqur41k/prj_f1j47h9srml4cyg962id75ms2e/workspaces/expwrk_k1d9n8z4panjhy1yh7bqe1ywiz?workspace-tab=dependencies


In [3]:
# Verify installation
import torch
import ray
import deepspeed

print(f"PyTorch version: {torch.__version__}")
print(f"Ray version: {ray.__version__}")
print(f"DeepSpeed version: {deepspeed.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

[2026-02-01 17:27:33,244] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.


/home/ray/anaconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


PyTorch version: 2.10.0+cu128
Ray version: 2.49.1
DeepSpeed version: 0.18.5
CUDA available: False


In [4]:
# Enable Ray Train V2 API
import os
os.environ["RAY_TRAIN_V2_ENABLED"] = "1"

# Standard imports
import tempfile
import uuid
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## `Step 2`: Model Definition (Same as FSDP2)

We use the **same Vision Transformer model** as the FSDP2 tutorial. This makes it easy to compare the two approaches directly. The key difference is that DeepSpeed wraps your existing PyTorch model, so no model code changes are needed.

In [5]:
import torch
from torchvision.models import VisionTransformer
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose

def init_model() -> torch.nn.Module:
    """Initialize a Vision Transformer model for FashionMNIST.
    
    This is IDENTICAL to the FSDP2 tutorial - no changes needed!
    DeepSpeed wraps your existing PyTorch model.
    """
    logger.info("Initializing Vision Transformer model...")

    model = VisionTransformer(
        image_size=28,
        patch_size=7,
        num_layers=10,
        num_heads=2,
        hidden_dim=128,
        mlp_dim=128,
        num_classes=10,
    )

    # Modify for grayscale input
    model.conv_proj = torch.nn.Conv2d(
        in_channels=1,
        out_channels=128,
        kernel_size=7,
        stride=7,
    )

    return model

# Verify model
test_model = init_model()
param_count = sum(p.numel() for p in test_model.parameters())
print(f"Model parameters: {param_count:,} ({param_count / 1e6:.2f}M)")
del test_model

INFO:__main__:Initializing Vision Transformer model...


Model parameters: 1,006,090 (1.01M)


## `Step 3`: DeepSpeed Configuration

**This is the key difference from FSDP2!** Instead of Python API calls, DeepSpeed uses a configuration dictionary (or JSON file). Every configuration option is highlighted and explained with a numbered comment; for example, "[1]."

### Configuration Anatomy

DeepSpeed configuration is declarative - you specify what you want, and DeepSpeed handles the implementation:

```python
deepspeed_config = {
    # Optimizer configuration (DeepSpeed manages the optimizer)
    "optimizer": {...},
    
    # Learning rate scheduler
    "scheduler": {...},
    
    # Mixed precision settings
    "fp16": {...} or "bf16": {...},
    
    # ZeRO optimization settings
    "zero_optimization": {
        "stage": 1|2|3,
        "offload_optimizer": {...},  # CPU offloading
        "offload_param": {...},      # Parameter offloading
    },
    
    # Batch size configuration
    "train_micro_batch_size_per_gpu": int,
    "gradient_accumulation_steps": int,
}
```

In [6]:
def get_deepspeed_config(batch_size: int = 64, learning_rate: float = 0.001) -> dict:
    """Create DeepSpeed configuration for ZeRO Stage 2.
    
    We use Stage 2 here as it provides a good balance between
    memory savings and communication overhead for our model size.
    
    For larger models (billions of parameters), use Stage 3.
    
    Args:
        batch_size: Micro batch size per GPU
        learning_rate: Initial learning rate
        
    Returns:
        DeepSpeed configuration dictionary
    """
    return {
        # [1] Optimizer Configuration.
        # DeepSpeed creates and manages the optimizer internally.
        # This replaces: optimizer = Adam(model.parameters(), lr=...)
        # ============================================================
        "optimizer": {
            "type": "Adam",
            "params": {
                "lr": learning_rate,
                "betas": [0.9, 0.999],
                "eps": 1e-8,
                "weight_decay": 0.0,
            }
        },
        
        # [2] Learning Rate Scheduler.
        # Optional: DeepSpeed can manage LR scheduling.
        # ==============================================
        "scheduler": {
            "type": "WarmupLR",
            "params": {
                "warmup_min_lr": 0,
                "warmup_max_lr": learning_rate,
                "warmup_num_steps": 100,
            }
        },
        
        # [3] Mixed Precision (FP16).
        # Equivalent to FSDP2's MixedPrecisionPolicy.
        # Provides ~2x memory reduction and faster computation.
        # ======================================================
        "fp16": {
            "enabled": True,
            "loss_scale": 0,              # Dynamic loss scaling
            "loss_scale_window": 1000,
            "initial_scale_power": 16,
            "hysteresis": 2,
            "min_loss_scale": 1,
        },
        
        # [4] ZeRO Optimization.
        # This is the core memory optimization feature.
        # Stage 2 partitions optimizer states + gradients.
        # =================================================
        "zero_optimization": {
            "stage": 2,  # Partition optimizer states + gradients
            
            # CPU Offloading (similar to FSDP2's CPUOffloadPolicy)
            # Uncomment to enable:
            # "offload_optimizer": {
            #     "device": "cpu",
            #     "pin_memory": True,
            # },
            
            # [5] Gradient bucketing for communication efficiency.
            # ======================================================
            "allgather_bucket_size": 2e8,
            "reduce_bucket_size": 2e8,
            
            # [6] Overlap communication with computation.
            # ===========================================
            "overlap_comm": True,
            
            # [7] Contiguous gradients for efficiency.
            # =========================================
            "contiguous_gradients": True,
        },
        
        # [8] Batch Size Configuration.
        # DeepSpeed uses these to calculate effective batch size.
        # =========================================================
        "train_micro_batch_size_per_gpu": batch_size,
        "gradient_accumulation_steps": 1,
        
        # [9] Gradient Clipping.
        # ======================
        "gradient_clipping": 1.0,
        
        # [10] Logging.
        # =============
        "steps_per_print": 100,
        "wall_clock_breakdown": False,
    }

# Preview the configuration
import json
print("DeepSpeed Configuration:")
print(json.dumps(get_deepspeed_config(), indent=2))

DeepSpeed Configuration:
{
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 0.001,
      "betas": [
        0.9,
        0.999
      ],
      "eps": 1e-08,
      "weight_decay": 0.0
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 0.001,
      "warmup_num_steps": 100
    }
  },
  "fp16": {
    "enabled": true,
    "loss_scale": 0,
    "loss_scale_window": 1000,
    "initial_scale_power": 16,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "zero_optimization": {
    "stage": 2,
    "allgather_bucket_size": 200000000.0,
    "reduce_bucket_size": 200000000.0,
    "overlap_comm": true,
    "contiguous_gradients": true
  },
  "train_micro_batch_size_per_gpu": 64,
  "gradient_accumulation_steps": 1,
  "gradient_clipping": 1.0,
  "steps_per_print": 100,
  "wall_clock_breakdown": false
}


## `Step 4`: DeepSpeed Checkpointing

DeepSpeed has **built-in checkpointing** that's simpler than FSDP2's DCP approach. The checkpointing methods are built into the DeepSpeed engine, so you don't need wrapper classes.

### Key Differences from FSDP2

| Feature | FSDP2 (DCP) | DeepSpeed |
|---------|-------------|------------|
| Save API | `dcp.save(state_dict, checkpoint_id)` | `model.save_checkpoint(dir)` |
| Load API | `dcp.load(state_dict, checkpoint_id)` | `model.load_checkpoint(dir)` |
| Format | `.distcp` shards | `mp_rank_*` directories |
| Resharding | Automatic on load | Automatic on load |
| Client code | Explicit wrapper needed | Built into engine |

In [7]:
import ray.train
import torch.distributed as dist

def save_deepspeed_checkpoint(
    model,  # DeepSpeed engine
    metrics: dict,
    epoch: int,
) -> None:
    """Save DeepSpeed checkpoint and report to Ray Train.
    
    Unlike FSDP2's DCP, DeepSpeed has built-in checkpoint methods.
    Each worker saves its shard, then we report to Ray Train.
    
    Args:
        model: DeepSpeed engine (not raw PyTorch model)
        metrics: Training metrics to report
        epoch: Current epoch number
    """
    logger.info("Saving DeepSpeed checkpoint...")
    
    with tempfile.TemporaryDirectory() as temp_dir:
        # DeepSpeed's built-in checkpoint method
        # This saves optimizer states, model weights, and scheduler
        model.save_checkpoint(
            save_dir=temp_dir,
            tag=f"epoch_{epoch}",
            client_state={"epoch": epoch},  # Custom metadata
        )
        
        # Synchronize all workers before reporting
        dist.barrier()
        
        # Report checkpoint to Ray Train
        checkpoint = ray.train.Checkpoint.from_directory(temp_dir)
        ray.train.report(metrics, checkpoint=checkpoint)
        
    logger.info(f"Checkpoint saved. Metrics: {metrics}")


def load_deepspeed_checkpoint(
    model,  # DeepSpeed engine
    checkpoint: ray.train.Checkpoint,
) -> int:
    """Load DeepSpeed checkpoint for resuming training.
    
    Args:
        model: DeepSpeed engine
        checkpoint: Ray Train checkpoint object
        
    Returns:
        Epoch number from checkpoint
    """
    logger.info("Loading DeepSpeed checkpoint...")
    
    with checkpoint.as_directory() as checkpoint_dir:
        # Find the checkpoint tag (e.g., "epoch_4")
        import os
        tags = [d for d in os.listdir(checkpoint_dir) 
                if d.startswith("epoch_")]
        if not tags:
            raise ValueError("No checkpoint found")
        
        latest_tag = sorted(tags)[-1]
        
        # DeepSpeed's built-in load method
        _, client_state = model.load_checkpoint(
            load_dir=checkpoint_dir,
            tag=latest_tag,
        )
        
    epoch = client_state.get("epoch", 0) if client_state else 0
    logger.info(f"Loaded checkpoint from epoch {epoch}")
    return epoch

In [8]:
from torch.distributed.checkpoint.state_dict import (
    get_model_state_dict,
    StateDictOptions
)

def save_model_for_inference(model, world_rank: int) -> None:
    """Save consolidated model for inference.
    
    For DeepSpeed, we can use the same approach as FSDP2:
    gather weights to rank 0 and save a standard checkpoint.
    
    Note: For ZeRO-3, use `zero.GatheredParameters` context manager.
    For ZeRO-1/2, the model weights are already complete on each rank.
    """
    logger.info("Saving model for inference...")
    
    with tempfile.TemporaryDirectory() as temp_dir:
        save_file = os.path.join(temp_dir, "full-model.pt")
        
        # For ZeRO Stage 2, weights are complete on each rank
        # We just need rank 0 to save
        if world_rank == 0:
            # Get the underlying PyTorch module
            state_dict = model.module.state_dict()
            torch.save(state_dict, save_file)
            logger.info(f"Saved model to {save_file}")
            checkpoint = ray.train.Checkpoint.from_directory(temp_dir)
        else:
            checkpoint = None
        
        # Synchronize before reporting
        dist.barrier()
        
        ray.train.report({}, checkpoint=checkpoint, checkpoint_dir_name="full_model")

## `Step 5`: Training Function

Here's the main training function. Notice the key differences from FSDP2. Every difference is highlighted and explained with a numbered comment; for example, "[1]."

### Key Differences from FSDP2:

1. **[1] No `prepare_model()`**: DeepSpeed handles model distribution internally
2. **[2] `deepspeed.initialize()`**: Creates the DeepSpeed engine and manages the optimizer
3. **[3] `model.backward(loss)`**: Instead of `loss.backward()` - DeepSpeed handles backward pass and zero_grad
4. **[4] `model.step()`**: Instead of `optimizer.step()` - DeepSpeed handles optimizer step

In [9]:
import ray.train.torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import deepspeed

def train_func(config):
    """DeepSpeed training function for Ray Train.
    
    Key differences from FSDP2:
    - Use deepspeed.initialize() instead of fully_shard()
    - DeepSpeed manages optimizer internally
    - Use model.backward(loss) instead of loss.backward()
    - Use model.step() instead of optimizer.step()
    """
    # ===== Model Setup =====
    model = init_model()
    
    # Get DeepSpeed configuration
    ds_config = get_deepspeed_config(
        batch_size=config.get('batch_size', 64),
        learning_rate=config.get('learning_rate', 0.001),
    )
    
    # [1] DeepSpeed Initialization.
    # This replaces:
    #   - model.to(device)
    #   - fully_shard(model, ...)
    #   - optimizer = Adam(model.parameters())
    # DeepSpeed handles model distribution and optimizer creation internally.
    # ======================================================================
    model_engine, optimizer, _, _ = deepspeed.initialize(
        model=model,
        config=ds_config,
        model_parameters=model.parameters(),
    )
    
    # Get the device from DeepSpeed
    device = model_engine.device
    
    # ===== Loss Function =====
    criterion = CrossEntropyLoss()
    
    # ===== Checkpoint Loading =====
    start_epoch = 0
    loaded_checkpoint = ray.train.get_checkpoint()
    if loaded_checkpoint:
        start_epoch = load_deepspeed_checkpoint(model_engine, loaded_checkpoint) + 1
        logger.info(f"Resuming from epoch {start_epoch}")
    
    # [2] Data Loading.
    # NOTE: With DeepSpeed, DON'T use ray.train.torch.prepare_data_loader()
    # DeepSpeed handles data distribution differently - use DistributedSampler directly.
    # ==================================================================================
    transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
    data_dir = os.path.join(tempfile.gettempdir(), "data")
    train_data = FashionMNIST(
        root=data_dir, train=True, download=True, transform=transform
    )
    
    # Create sampler for distributed training
    sampler = torch.utils.data.DistributedSampler(
        train_data,
        num_replicas=ray.train.get_context().get_world_size(),
        rank=ray.train.get_context().get_world_rank(),
        shuffle=True,
    )
    
    train_loader = DataLoader(
        train_data,
        batch_size=config.get('batch_size', 64),
        sampler=sampler,
        num_workers=2,
        pin_memory=True,
    )
    
    # ===== Training Context =====
    world_rank = ray.train.get_context().get_world_rank()
    run_name = ray.train.get_context().get_experiment_name()
    
    # ===== Memory Profiling =====
    torch.cuda.memory._record_memory_history(max_entries=100000)
    
    # ===== Training Loop =====
    running_loss = 0.0
    num_batches = 0
    epochs = config.get('epochs', 5)
    
    for epoch in range(start_epoch, epochs):
        sampler.set_epoch(epoch)  # For proper shuffling
        
        for images, labels in train_loader:
            # Move data to device
            images = images.to(device)
            labels = labels.to(device)
            
            # IMPORTANT: Cast inputs to FP16 when using mixed precision
            # DeepSpeed converts model weights to FP16, so inputs must match
            if model_engine.fp16_enabled():
                images = images.half()
            
            # Forward pass
            outputs = model_engine(images)
            loss = criterion(outputs, labels)
            
            # [3] Backward pass - DeepSpeed style!
            # This replaces:
            #   optimizer.zero_grad()
            #   loss.backward()
            #   optimizer.step()
            # DeepSpeed handles all three operations internally.
            # ====================================================
            model_engine.backward(loss)
            model_engine.step()
            
            running_loss += loss.item()
            num_batches += 1
        
        # Report metrics and save checkpoint
        avg_loss = running_loss / num_batches
        metrics = {"loss": avg_loss, "epoch": epoch}
        save_deepspeed_checkpoint(model_engine, metrics, epoch)
        
        if world_rank == 0:
            logger.info(f"Epoch {epoch}: loss={avg_loss:.4f}")
    
    # ===== Save Memory Snapshot =====
    try:
        snapshot_path = f"/mnt/cluster_storage/{run_name}/rank{world_rank}_memory_snapshot.pickle"
        torch.cuda.memory._dump_snapshot(snapshot_path)
        logger.info(f"Saved memory snapshot to {snapshot_path}")
    except Exception as e:
        logger.warning(f"Could not save memory snapshot: {e}")
    finally:
        torch.cuda.memory._record_memory_history(enabled=None)
    
    # ===== Save Final Model =====
    save_model_for_inference(model_engine, world_rank)

## `Step 6`: Launch Distributed Training

The Ray Train setup is **almost identical** to FSDP2. The main difference is that we don't need any special FSDP-specific configuration - DeepSpeed handles everything through its config dict.

In [10]:
import ray.train
import ray.train.torch

# Scaling configuration - same as FSDP2!
scaling_config = ray.train.ScalingConfig(
    num_workers=2,
    use_gpu=True,
)

# Training hyperparameters
train_loop_config = {
    "epochs": 5,
    "learning_rate": 0.001,
    "batch_size": 64,
}

# Unique experiment name
experiment_name = f"deepspeed_mnist_{uuid.uuid4().hex[:8]}"

# Run configuration
run_config = ray.train.RunConfig(
    storage_path="/mnt/cluster_storage/",
    name=experiment_name,
    failure_config=ray.train.FailureConfig(max_failures=1),
)

print(f"Experiment: {experiment_name}")
print(f"Workers: {scaling_config.num_workers}")
print(f"Epochs: {train_loop_config['epochs']}")

Experiment: deepspeed_mnist_34e40726
Workers: 2
Epochs: 5


In [11]:
# Create and run trainer
trainer = ray.train.torch.TorchTrainer(
    train_loop_per_worker=train_func,
    scaling_config=scaling_config,
    train_loop_config=train_loop_config,
    run_config=run_config,
)

print("Starting DeepSpeed training...")
result = trainer.fit()
print("\nTraining completed!")
print(f"Final checkpoint: {result.checkpoint}")

2026-02-01 17:27:38,747	INFO worker.py:1771 -- Connecting to existing Ray cluster at address: 10.0.127.246:6379...


Starting DeepSpeed training...


2026-02-01 17:27:38,758	INFO worker.py:1942 -- Connected to Ray cluster. View the dashboard at https://session-lehzcwqlg8w8ui213vjr5igw7j.i.anyscaleuserdata.com 
2026-02-01 17:27:38,761	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_53c05f65135986a0fc4a81b263296a532a2a3021.zip' (0.61MiB) to Ray cluster...
2026-02-01 17:27:38,765	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_53c05f65135986a0fc4a81b263296a532a2a3021.zip'.
(TrainController pid=18125) [State Transition] INITIALIZING -> SCHEDULING.
(TrainController pid=18125) Attempting to start training worker group of size 2 with the following resources: [{'GPU': 1}] * 2
(TrainController pid=18125) Using blocking ray.get inside async actor. This blocks the event loop. Please use `await` on object ref with asyncio.gather if you want to yield execution to the event loop instead.
(RayTrainWorker pid=6819, ip=10.0.102.145) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pi

(RayTrainWorker pid=7619, ip=10.0.82.245) [2026-02-01 17:27:51,599] [WARNING] [lr_schedules.py:690:get_lr] Attempting to get learning rate from scheduler before it has started


(RayTrainWorker pid=6819, ip=10.0.102.145) Before initializing optimizer states
(RayTrainWorker pid=6819, ip=10.0.102.145) MA 0.0 GB         Max_MA 0.0 GB         CA 0.02 GB         Max_CA 0 GB 
(RayTrainWorker pid=6819, ip=10.0.102.145) CPU Virtual Memory:  used = 3.46 GB, percent = 11.2%
(RayTrainWorker pid=6819, ip=10.0.102.145) After initializing optimizer states
(RayTrainWorker pid=6819, ip=10.0.102.145) MA 0.0 GB         Max_MA 0.01 GB         CA 0.02 GB         Max_CA 0 GB 
(RayTrainWorker pid=6819, ip=10.0.102.145) CPU Virtual Memory:  used = 3.46 GB, percent = 11.2%
(RayTrainWorker pid=6819, ip=10.0.102.145) After initializing ZeRO optimizer
(RayTrainWorker pid=6819, ip=10.0.102.145) MA 0.0 GB         Max_MA 0.0 GB         CA 0.02 GB         Max_CA 0 GB 
(RayTrainWorker pid=6819, ip=10.0.102.145) CPU Virtual Memory:  used = 3.46 GB, percent = 11.2%


(autoscaler +49s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


(RayTrainWorker pid=6819, ip=10.0.102.145) Saving DeepSpeed checkpoint...
(RayTrainWorker pid=6819, ip=10.0.102.145) /tmp/ray/session_2026-02-01_16-45-46_338165_2431/runtime_resources/pip/ae159e3c0955bfdbdfe1ff48e527ea80167311c2/virtualenv/lib/python3.12/site-packages/torch/distributed/c10d_logger.py:83: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
(RayTrainWorker pid=6819, ip=10.0.102.145)   return func(*args, **kwargs)
(RayTrainWorker pid=6819, ip=10.0.102.145) [rank0]:[W201 17:28:55.997155180 ProcessGroupNCCL.cpp:5138] Guessing device ID based on global rank. This can cause a hang if rank to GPU mapping is heterogeneous. You can specify device_id in init_process_group()
(RayTrainWorker pid=6819, ip=10.0.102.145) Initializing Vision Transformer model...
(RayTrainWorker pid=7619, ip=10.0.82.245) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/mnt/cluster_storage/deepspee


Training completed!
Final checkpoint: Checkpoint(filesystem=local, path=/mnt/cluster_storage/deepspeed_mnist_34e40726/full_model)


## `Step 7`: Inspect Training Artifacts

DeepSpeed checkpoint structure differs from FSDP2's DCP format:

```
/mnt/cluster_storage/{experiment_name}/
├── checkpoint_*/
│   └── epoch_N/                  # DeepSpeed checkpoint tag
│       ├── mp_rank_00_model_states.pt  # Model states for rank 0
│       ├── mp_rank_01_model_states.pt  # Model states for rank 1
│       ├── zero_pp_rank_0_mp_rank_00_optim_states.pt
│       └── zero_pp_rank_1_mp_rank_01_optim_states.pt
├── full_model/
│   └── full-model.pt
└── rank*_memory_snapshot.pickle
```

In [12]:
# List training artifacts
storage_path = f"/mnt/cluster_storage/{experiment_name}/"
print(f"Artifacts in {storage_path}:")
for item in os.listdir(storage_path):
    full_path = os.path.join(storage_path, item)
    if os.path.isdir(full_path):
        print(f"  📁 {item}/")
        # Show DeepSpeed checkpoint structure
        if item.startswith("checkpoint"):
            for sub in os.listdir(full_path)[:3]:  # Limit output
                print(f"      └── {sub}")
    else:
        size_mb = os.path.getsize(full_path) / (1024 * 1024)
        print(f"  📄 {item} ({size_mb:.2f} MB)")

Artifacts in /mnt/cluster_storage/deepspeed_mnist_34e40726/:
  📄 rank1_memory_snapshot.pickle (18.76 MB)
  📁 checkpoint_2026-02-01_17-32-05.953257/
      └── epoch_3
      └── latest
      └── zero_to_fp32.py
  📄 rank0_memory_snapshot.pickle (18.77 MB)
  📄 checkpoint_manager_snapshot.json (0.00 MB)
  📄 .validate_storage_marker (0.00 MB)
  📁 checkpoint_2026-02-01_17-29-59.126707/
      └── latest
      └── zero_to_fp32.py
      └── epoch_1
  📁 checkpoint_2026-02-01_17-33-09.432923/
      └── epoch_4
      └── latest
      └── zero_to_fp32.py
  📁 full_model/
  📁 checkpoint_2026-02-01_17-31-02.544201/
      └── epoch_2
      └── latest
      └── zero_to_fp32.py
  📁 checkpoint_2026-02-01_17-28-55.856021/
      └── epoch_0
      └── latest
      └── zero_to_fp32.py


## `Step 8`: Load Model for Inference

Loading for inference is **identical** to FSDP2 - we saved a standard PyTorch checkpoint.

In [13]:
# Path to saved model
PATH_TO_FULL_MODEL = f"/mnt/cluster_storage/{experiment_name}/full_model/full-model.pt"
print(f"Loading model from: {PATH_TO_FULL_MODEL}")

Loading model from: /mnt/cluster_storage/deepspeed_mnist_34e40726/full_model/full-model.pt


In [14]:
# Load trained model
inference_model = init_model()
state_dict = torch.load(PATH_TO_FULL_MODEL, map_location='cpu', weights_only=True)
inference_model.load_state_dict(state_dict)
inference_model.eval()
print("Model loaded successfully!")

INFO:__main__:Initializing Vision Transformer model...


Model loaded successfully!


In [15]:
# Test inference
transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
test_data = FashionMNIST(root="/tmp", train=False, download=True, transform=transform)

CLASSES = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

print("\nSample predictions:")
print("-" * 50)

with torch.no_grad():
    correct = 0
    for i in range(10):
        image = test_data.data[i].reshape(1, 1, 28, 28).float()
        output = inference_model(image)
        predicted = output.argmax().item()
        actual = test_data.targets[i].item()
        correct += (predicted == actual)
        
        status = "✓" if predicted == actual else "✗"
        print(f"{status} Sample {i}: predicted={CLASSES[predicted]:12s} actual={CLASSES[actual]}")
    
    print("-" * 50)
    print(f"Accuracy on samples: {correct}/10 ({correct*10}%)")


Sample predictions:
--------------------------------------------------
✗ Sample 0: predicted=Pullover     actual=Ankle boot
✓ Sample 1: predicted=Pullover     actual=Pullover
✗ Sample 2: predicted=Bag          actual=Trouser
✗ Sample 3: predicted=T-shirt/top  actual=Trouser
✗ Sample 4: predicted=Pullover     actual=Shirt
✗ Sample 5: predicted=Shirt        actual=Trouser
✓ Sample 6: predicted=Coat         actual=Coat
✗ Sample 7: predicted=Coat         actual=Shirt
✗ Sample 8: predicted=Pullover     actual=Sandal
✗ Sample 9: predicted=Pullover     actual=Sneaker
--------------------------------------------------
Accuracy on samples: 2/10 (20%)


## Summary: FSDP2 vs DeepSpeed

### Side-by-Side Comparison

| Aspect | FSDP2 | DeepSpeed |
|--------|-------|----------|
| **Setup** | `fully_shard(model, mesh, ...)` | `deepspeed.initialize(model, config)` |
| **Optimizer** | User creates separately | Managed by DeepSpeed |
| **Training Step** | `loss.backward()` | `model.backward(loss)` |
| **Optimizer Step** | `optimizer.step()` | `model.step()` |
| **Configuration** | Python API | JSON/dict config |
| **Checkpointing** | PyTorch DCP | Built-in methods |
| **Data Loader** | `prepare_data_loader()` | Manual `DistributedSampler` |

### When to Use Which?

**Choose FSDP2 when:**
- You want native PyTorch integration
- You're using PyTorch's ecosystem (DTensor, etc.)
- You prefer Python API over config files
- You need fine-grained control over sharding

**Choose DeepSpeed when:**
- Training very large models (billions of parameters)
- You need advanced CPU/NVMe offloading
- Using HuggingFace Transformers (great integration)
- You want battle-tested LLM training recipes
- You need fused optimizers (FusedAdam, etc.)

### Key Takeaways

1. **Both achieve similar memory savings** through different APIs
2. **Ray Train works seamlessly** with both approaches
3. **Model definition stays the same** - only the wrapping changes
4. **Inference is identical** - save a standard PyTorch checkpoint

### Resources

- [DeepSpeed Documentation](https://www.deepspeed.ai/)
- [DeepSpeed ZeRO Tutorial](https://www.deepspeed.ai/tutorials/zero/)
- [Ray Train DeepSpeed Guide](https://docs.ray.io/en/latest/train/deepspeed.html)
- [Ray Train DeepSpeed Example](https://docs.ray.io/en/latest/train/examples/deepspeed/deepspeed_example.html)

## 10. Bonus: ZeRO Stage 3 Configuration

For reference, here's how to configure ZeRO Stage 3 (equivalent to FSDP2 with `reshard_after_forward=True`):

In [16]:
def get_zero3_config(batch_size: int = 64, learning_rate: float = 0.001) -> dict:
    """ZeRO Stage 3 configuration with CPU offloading.
    
    This is the maximum memory optimization - equivalent to:
    - FSDP2 with reshard_after_forward=True
    - FSDP2 with CPUOffloadPolicy()
    - FSDP2 with MixedPrecisionPolicy()
    """
    return {
        "optimizer": {
            "type": "AdamW",
            "params": {"lr": learning_rate, "weight_decay": 0.01}
        },
        
        "fp16": {"enabled": True},
        
        "zero_optimization": {
            "stage": 3,  # Full parameter partitioning!
            
            # CPU offloading for optimizer states
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True,
            },
            
            # CPU offloading for parameters
            "offload_param": {
                "device": "cpu",
                "pin_memory": True,
            },
            
            # Memory optimization flags
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": 5e7,
            "stage3_prefetch_bucket_size": 5e7,
            "stage3_param_persistence_threshold": 1e5,
            
            # Gather weights for saving/loading
            "stage3_gather_16bit_weights_on_model_save": True,
        },
        
        "train_micro_batch_size_per_gpu": batch_size,
        "gradient_accumulation_steps": 1,
        "gradient_clipping": 1.0,
    }

print("ZeRO Stage 3 Configuration (for large models):")
print(json.dumps(get_zero3_config(), indent=2))

ZeRO Stage 3 Configuration (for large models):
{
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 0.001,
      "weight_decay": 0.01
    }
  },
  "fp16": {
    "enabled": true
  },
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    },
    "offload_param": {
      "device": "cpu",
      "pin_memory": true
    },
    "overlap_comm": true,
    "contiguous_gradients": true,
    "reduce_bucket_size": 50000000.0,
    "stage3_prefetch_bucket_size": 50000000.0,
    "stage3_param_persistence_threshold": 100000.0,
    "stage3_gather_16bit_weights_on_model_save": true
  },
  "train_micro_batch_size_per_gpu": 64,
  "gradient_accumulation_steps": 1,
  "gradient_clipping": 1.0
}


---

**Next Steps for the Blog:**

1. Run both notebooks on the same cluster
2. Compare memory usage profiles
3. Benchmark training throughput
4. Try with larger models (GPT-2, BERT)

Both FSDP2 and DeepSpeed are excellent choices for distributed training. The best choice depends on your specific use case, existing tooling, and team familiarity.